# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""../../../../da…","""13162532141526…","""16393585128650…",1.2927e19,"""17967518745389…","""10667307397135…",false
"""../../../../da…","""22327275621277…","""80738214335114…",1.4856e19,"""17967518745389…","""91349648356075…",true
"""../../../../da…","""71267959666747…","""22308464403280…",5.6458e17,"""14413288701990…","""10667307397135…",false
"""../../../../da…","""22327275621277…","""63537247226430…",4.0723e18,"""31854394385421…","""10667307397135…",false
"""../../../../da…","""13162532141526…","""14569052436926…",1.4677e19,"""31854394385421…","""16597108704203…",true
"""../../../../da…","""71267959666747…","""11583844034896…",1.6330e19,"""16513521720234…",null,false
"""../../../../da…","""71267959666747…","""12714244815129…",8.7675e18,"""31854394385421…","""16597108704203…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../da…","""14346263775424…","""15962270862107…",1.2927e19,"""FirstMortgage3…","""33910925400977…","""Rejected"""
"""../../../../da…","""93548594396485…","""13001260806302…",1.4856e19,"""FirstMortgage3…","""81623812645215…","""Accepted"""
"""../../../../da…","""82369845046256…","""15934768941265…",5.6458e17,"""MoneyMarketSav…","""33910925400977…","""Rejected"""
"""../../../../da…","""93548594396485…","""11756320322906…",4.0723e18,"""BasicChecking""","""33910925400977…","""Rejected"""
"""../../../../da…","""14346263775424…","""24720120222833…",1.4677e19,"""BasicChecking""","""71312238589880…","""Accepted"""
"""../../../../da…","""82369845046256…","""17193908736011…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""82369845046256…","""16526790954226…",8.7675e18,"""BasicChecking""","""71312238589880…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../da…","""12052365626173…","""16014625708796…",1.2927e19,"""FirstMortgage3…","""10032699116207…","""Rejected"""
"""../../../../da…","""66190943918933…","""12587381290624…",1.4856e19,"""FirstMortgage3…","""15071511925841…","""Accepted"""
"""../../../../da…","""15694079305192…","""15884553225396…",5.6458e17,"""MoneyMarketSav…","""10032699116207…","""Rejected"""
"""../../../../da…","""66190943918933…","""22560853359593…",4.0723e18,"""BasicChecking""","""10032699116207…","""Rejected"""
"""../../../../da…","""12052365626173…","""79043896039421…",1.4677e19,"""BasicChecking""","""10704517234339…","""Accepted"""
"""../../../../da…","""15694079305192…","""29156544383692…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""15694079305192…","""20065452508901…",8.7675e18,"""BasicChecking""","""10704517234339…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_MaritalStatus=PREDICTOR_1
Customer_City=PREDICTOR_2
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""97800958896705…","""94634722308245…",1.2927e19,"""FirstMortgage3…","""65939190003388…","""Rejected"""
"""17345172212663…","""48704912334109…",1.4856e19,"""FirstMortgage3…","""11208405990540…","""Accepted"""
"""15463151947305…","""15835845628485…",5.6458e17,"""MoneyMarketSav…","""65939190003388…","""Rejected"""
"""17345172212663…","""94136447049391…",4.0723e18,"""BasicChecking""","""65939190003388…","""Rejected"""
"""97800958896705…","""68239903795898…",1.4677e19,"""BasicChecking""","""56989559042602…","""Accepted"""
"""15463151947305…","""14595392990158…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""15463151947305…","""75542362386650…",8.7675e18,"""BasicChecking""","""56989559042602…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'